In [1]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Rescaling,
    Conv2D,
    MaxPooling2D,
    AveragePooling2D,
    Flatten,
    Dense,
    Input,
    BatchNormalization,
)
from tensorflow.keras import Model
import numpy as np
import tqdm
import itertools
import random
import uuid
import json
import os

2023-08-03 20:30:46.662808: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 20:30:47.788221: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Check if GPU available
tf.config.list_physical_devices('GPU')

2023-08-03 20:30:49.105563: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-03 20:30:49.238669: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-03 20:30:49.238711: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
BATCH_SIZE = 32
IMG_HEIGHT = 128
IMG_WIDTH = 128
NUM_CLASSES = 716
RANDOM_SEED = 0

DATA_PATH = "../data/derived_data/data_augmented/"
RESULTS_PATH = "./hps_search_results/"

In [4]:
def set_random_seed(seed):
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    
set_random_seed(RANDOM_SEED)

In [5]:
hps_grid = dict(
    conv2d_1_filters = [16, 32, 64],
    conv2d_1_kernel = [3, 5],
    conv2d_2_filters = [32, 64, 128],
    conv2d_2_kernel = [3, 5],
    conv2d_3_filters = [64, 128, 256],
    conv2d_3_kernel = [3,5],
    dense_1_units = [128, 256, 512, 1024],
    pooling = [
        MaxPooling2D,
        AveragePooling2D,
    ],
    batch_normalization = [False, True],
    optimizer = [
        tf.keras.optimizers.Adam,
        tf.keras.optimizers.RMSprop,
    ],
    learning_rate = [0.001, 0.005]
)

In [6]:
params = list(itertools.product(*hps_grid.values()))
hps_combs = [dict(zip(hps_grid.keys(),params_sample)) for params_sample in params]

In [7]:
filtered_combs = []
for comb in tqdm.tqdm(hps_combs):
    if comb["conv2d_1_filters"] <= comb["conv2d_2_filters"] and comb["conv2d_2_filters"] <= comb["conv2d_3_filters"]:
        filtered_combs.append(comb)

100%|██████████| 13824/13824 [00:00<00:00, 2789208.12it/s]


In [8]:
hps_combs = filtered_combs
len(hps_combs)

10752

In [9]:
random.shuffle(hps_combs)

In [10]:
def define_model(hps_comb):
    
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    x = Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))(inputs)
    
    x = Conv2D(hps_comb['conv2d_1_filters'], hps_comb['conv2d_1_kernel'], padding='same', activation='relu')(x)
    x = hps_comb["pooling"]()(x)
    
    if hps_comb["batch_normalization"]:
        x = BatchNormalization()(x)
        
    x = Conv2D(hps_comb['conv2d_2_filters'], hps_comb['conv2d_2_kernel'], padding='same', activation='relu')(x)
    x = hps_comb["pooling"]()(x)
    
    if hps_comb["batch_normalization"]:
        x = BatchNormalization()(x)
        
    x = Conv2D(hps_comb['conv2d_3_filters'], hps_comb['conv2d_3_kernel'], padding='same', activation='relu')(x)
    x = hps_comb["pooling"]()(x)
    
    if hps_comb["batch_normalization"]:
        x = BatchNormalization()(x)
        
    x = Flatten()(x)
    x = Dense(hps_comb['dense_1_units'], activation='relu')(x)
    outputs = Dense(NUM_CLASSES)(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="")

    model.compile(
        optimizer=hps_comb['optimizer'](learning_rate=hps_comb['learning_rate']),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )

    return model


def train_model(
        model,
        train_ds,
        val_ds,
        test_ds,
        checkpoint_filepath=None, 
        epochs=5
    ):

    # Define callbacks
    # early_stopping = tf.keras.callbacks.EarlyStopping(
    #     monitor='val_loss',
    #     patience=5,
    #     restore_best_weights=True,
    # )

    
    # model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    #     filepath=checkpoint_filepath,
    #     save_weights_only=True,
    #     monitor='val_accuracy',
    #     save_best_only=True
    # )
    
    # Set the same RANDOM_SEED for every model
    set_random_seed(RANDOM_SEED)
    
    # Train model
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=[
            # early_stopping,
            # model_checkpoint,
        ],
        verbose='auto',
    )
    
    results = history.history

    test_loss, test_acc = model.evaluate(test_ds)

    results['test_loss'] = test_loss
    results['test_acc'] = test_acc
    
    return results


def run_hps_search(train_ds, val_ds, test_ds, output_path):
    hps_search_id = uuid.uuid4()

    output_path = f'{output_path}/{hps_search_id}/'
    os.makedirs(output_path, exist_ok=True)

    for hps_comb in tqdm.tqdm(hps_combs):
        print(hps_comb)
        hps_comb_id = uuid.uuid4()

        model = define_model(hps_comb)

        results = train_model(
            model, 
            train_ds=train_ds,
            val_ds=val_ds,
            test_ds=test_ds,
        )

        for k,v in hps_comb.items():
            results[k] = str(v)

        with open(f'{output_path}/{hps_comb_id}.json', 'w') as f:
            json.dump(results, f)


In [11]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH,
    labels="inferred",
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    validation_split=0.3,
    subset="training",
    image_size=(IMG_WIDTH, IMG_HEIGHT),
    shuffle=True,
    seed=RANDOM_SEED,
    interpolation='nearest'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH,
    labels="inferred",
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    validation_split=0.3,
    subset="validation",
    image_size=(IMG_WIDTH, IMG_HEIGHT),
    shuffle=True,
    seed=RANDOM_SEED,
    interpolation='nearest'
)

val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 2)
val_ds = val_ds.skip(val_batches // 2)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 78044 files belonging to 716 classes.
Using 54631 files for training.


2023-08-03 20:31:39.704528: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-03 20:31:39.704610: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-03 20:31:39.704635: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-03 20:31:40.715945: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-03 20:31:40.715996: I tensorflow/compile

Found 78044 files belonging to 716 classes.
Using 23413 files for validation.


In [12]:
run_hps_search(train_ds, val_ds, test_ds, RESULTS_PATH)

  0%|          | 0/10752 [00:00<?, ?it/s]

{'conv2d_1_filters': 16, 'conv2d_1_kernel': 5, 'conv2d_2_filters': 128, 'conv2d_2_kernel': 3, 'conv2d_3_filters': 128, 'conv2d_3_kernel': 5, 'dense_1_units': 1024, 'pooling': <class 'keras.src.layers.pooling.max_pooling2d.MaxPooling2D'>, 'batch_normalization': False, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'learning_rate': 0.005}
Epoch 1/5


2023-08-03 20:31:50.090911: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-03 20:31:50.717028: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-03 20:31:51.376530: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x31e31330 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-03 20:31:51.376570: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660, Compute Capability 7.5
2023-08-03 20:31:51.386480: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-03 20:31:51.514316: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-03 20:31:51.563560: I ./tensorflow/compiler/jit/device_compiler

1708/1708 [==============================] - 75s 41ms/step - loss: 6.6303 - accuracy: 0.0012 - val_loss: 6.5846 - val_accuracy: 5.1278e-04
Epoch 2/5
1708/1708 [==============================] - 67s 39ms/step - loss: 6.5814 - accuracy: 0.0011 - val_loss: 6.5864 - val_accuracy: 5.1278e-04
Epoch 3/5
1708/1708 [==============================] - 66s 39ms/step - loss: 6.5817 - accuracy: 0.0012 - val_loss: 6.5869 - val_accuracy: 5.1278e-04
Epoch 4/5
1708/1708 [==============================] - 66s 39ms/step - loss: 6.5817 - accuracy: 9.1523e-04 - val_loss: 6.5867 - val_accuracy: 5.1278e-04
Epoch 5/5
366/366 [==============================] - 4s 9ms/step - loss: 6.5872 - accuracy: 6.8306e-04


  0%|          | 1/10752 [05:43<1025:17:02, 343.32s/it]

...

{'conv2d_1_filters': 32, 'conv2d_1_kernel': 3, 'conv2d_2_filters': 64, 'conv2d_2_kernel': 3, 'conv2d_3_filters': 128, 'conv2d_3_kernel': 3, 'dense_1_units': 512, 'pooling': <class 'keras.src.layers.pooling.max_pooling2d.MaxPooling2D'>, 'batch_normalization': False, 'optimizer': <class 'keras.src.optimizers.rmsprop.RMSprop'>, 'learning_rate': 0.005}
Epoch 1/5
1708/1708 [==============================] - 42s 24ms/step - loss: 6.6334 - accuracy: 0.0011 - val_loss: 6.5977 - val_accuracy: 0.0011
Epoch 2/5
1708/1708 [==============================] - 41s 24ms/step - loss: 6.5959 - accuracy: 0.0012 - val_loss: 6.6066 - val_accuracy: 5.1278e-04
Epoch 3/5
1708/1708 [==============================] - 41s 24ms/step - loss: 6.5968 - accuracy: 0.0010 - val_loss: 6.6041 - val_accuracy: 6.8370e-04
Epoch 4/5
1708/1708 [==============================] - 41s 24ms/step - loss: 6.5967 - accuracy: 0.0010 - val_loss: 6.6060 - val_accuracy: 5.1278e-04
Epoch 5/5
366/366 [==============================] - 3s 7

  0%|          | 23/10752 [1:47:49<898:05:23, 301.34s/it]

{'conv2d_1_filters': 64, 'conv2d_1_kernel': 3, 'conv2d_2_filters': 128, 'conv2d_2_kernel': 5, 'conv2d_3_filters': 128, 'conv2d_3_kernel': 3, 'dense_1_units': 1024, 'pooling': <class 'keras.src.layers.pooling.max_pooling2d.MaxPooling2D'>, 'batch_normalization': False, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'learning_rate': 0.001}
Epoch 1/5
1708/1708 [==============================] - 90s 52ms/step - loss: 4.2128 - accuracy: 0.2620 - val_loss: 2.2217 - val_accuracy: 0.5486
Epoch 2/5
1708/1708 [==============================] - 89s 52ms/step - loss: 1.3379 - accuracy: 0.6987 - val_loss: 1.6752 - val_accuracy: 0.6773
Epoch 3/5
1708/1708 [==============================] - 89s 52ms/step - loss: 0.5108 - accuracy: 0.8708 - val_loss: 2.1396 - val_accuracy: 0.6915
Epoch 4/5
1708/1708 [==============================] - 89s 52ms/step - loss: 0.2358 - accuracy: 0.9379 - val_loss: 2.3372 - val_accuracy: 0.6948
Epoch 5/5
366/366 [==============================] - 5s 15ms/step - loss:

  0%|          | 24/10752 [1:55:20<1032:03:32, 346.33s/it]

{'conv2d_1_filters': 32, 'conv2d_1_kernel': 5, 'conv2d_2_filters': 128, 'conv2d_2_kernel': 5, 'conv2d_3_filters': 256, 'conv2d_3_kernel': 5, 'dense_1_units': 512, 'pooling': <class 'keras.src.layers.pooling.average_pooling2d.AveragePooling2D'>, 'batch_normalization': True, 'optimizer': <class 'keras.src.optimizers.rmsprop.RMSprop'>, 'learning_rate': 0.005}
Epoch 1/5
1708/1708 [==============================] - 99s 57ms/step - loss: 11.3905 - accuracy: 0.0013 - val_loss: 6.6001 - val_accuracy: 5.1278e-04
Epoch 2/5
1385/1708 [=======================>......] - ETA: 17s - loss: 6.5995 - accuracy: 0.0014

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.axis("off")